In [1]:
# pip install openpyxl
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [2]:
# 1. 데이터 불러오기
# pandas로 엑셀 데이터를 불러옵니다.
# file_path는 불러올 엑셀 파일의 경로입니다.
# sheet_name은 불러올 시트의 이름을 지정하는데, 여기서는 '2023년 01월' 데이터를 가져옵니다.
# 참고: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_excel.html
file_path = '../data/2023년 01월 서울시 교통량.xlsx'
data = pd.read_excel(file_path, sheet_name="2023년 01월")

# 데이터를 살펴보면 현재 일자와 요일이 일치하지 않습니다. 그렇기 때문에 일자 칼럼을 이용하여 요일을 다시 계산해야 합니다.

In [3]:
data

,일자,요일,지점명,지점번호,방향,구분,0시,1시,2시,3시,...,14시,15시,16시,17시,18시,19시,20시,21시,22시,23시
0,20230101,일,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,564.0,414.0,268.0,208.0,...,1388.0,1370.0,1322.0,1263.0,979.0,875.0,785.0,824.0,552.0,421.0
1,20230102,월,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,294.0,190.0,146.0,134.0,...,1459.0,1571.0,1566.0,1792.0,1582.0,1248.0,997.0,973.0,872.0,616.0
2,20230103,화,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,419.0,271.0,203.0,177.0,...,1663.0,1678.0,1711.0,1851.0,1697.0,1265.0,1081.0,1090.0,883.0,757.0
3,20230104,수,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,502.0,335.0,247.0,192.0,...,1679.0,1798.0,1719.0,1807.0,1679.0,1240.0,1112.0,1069.0,909.0,769.0
4,20230105,목,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,523.0,334.0,235.0,196.0,...,1530.0,1627.0,1805.0,1798.0,1770.0,1289.0,1152.0,1111.0,1030.0,756.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8613,20230127,금,신월여의지하도로,F-10,유출,서울교->신월IC,503.0,361.0,208.0,140.0,...,1638.0,2131.0,2094.0,2022.0,2001.0,2049.0,2086.0,1737.0,1460.0,1129.0
8614,20230128,토,신월여의지하도로,F-10,유출,서울교->신월IC,630.0,433.0,287.0,217.0,...,1762.0,1845.0,1647.0,1801.0,2009.0,1736.0,1557.0,1455.0,1383.0,786.0
8615,20230129,일,신월여의지하도로,F-10,유출,서울교->신월IC,556.0,392.0,254.0,171.0,...,1515.0,1690.0,1732.0,1706.0,1561.0,1261.0,1366.0,1206.0,901.0,516.0
8616,20230130,월,신월여의지하도로,F-10,유출,서울교->신월IC,317.0,235.0,116.0,97.0,...,1508.0,1913.0,2156.0,2052.0,2059.0,2075.0,1928.0,1467.0,1313.0,823.0


In [4]:
# 2. '일자' 컬럼을 datetime 형식으로 변환
# 이를 datetime 형식으로 변환하면 날짜 관련 작업이 더 쉬워집니다.
# pandas의 to_datetime 함수를 사용하여 '일자' 데이터를 날짜 형식으로 변환합니다.
# (참고: '일자' 컬럼은 날짜를 나타내는 값인데, 이 컬럼의 각 값들은 정수 형식(YYYYMMDD)으로 되어 있습니다.)
# 정수 형식에 대한 format을 선언해줘야 합니다.
# 해주지 않으면 1970-01-01 00:00:00.020230101와 같은 형식으로 표현됩니다.
# 참고: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_datetime.html
data['일자'] = pd.to_datetime(data['일자'], format="%Y%m%d")

In [5]:
data['일자']

0      2023-01-01
1      2023-01-02
2      2023-01-03
3      2023-01-04
4      2023-01-05
          ...    
8613   2023-01-27
8614   2023-01-28
8615   2023-01-29
8616   2023-01-30
8617   2023-01-31
Name: 일자, Length: 8618, dtype: datetime64[ns]

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8618 entries, 0 to 8617
Data columns (total 30 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   일자      8618 non-null   datetime64[ns]
 1   요일      8618 non-null   object        
 2   지점명     8618 non-null   object        
 3   지점번호    8618 non-null   object        
 4   방향      8618 non-null   object        
 5   구분      8618 non-null   object        
 6   0시      6589 non-null   float64       
 7   1시      6588 non-null   float64       
 8   2시      6584 non-null   float64       
 9   3시      6569 non-null   float64       
 10  4시      6567 non-null   float64       
 11  5시      6566 non-null   float64       
 12  6시      6569 non-null   float64       
 13  7시      6570 non-null   float64       
 14  8시      6572 non-null   float64       
 15  9시      6576 non-null   float64       
 16  10시     6581 non-null   float64       
 17  11시     6588 non-null   float64       
 18  12시     

In [7]:
# 3. 날짜를 기반으로 요일을 자동으로 계산
# 요일이 정수로 변환되면 학습 모델에서 이를 숫자형 데이터로 처리할 수 있습니다.
# datetime 형식으로 변환된 '일자' 컬럼을 기반으로 요일을 계산합니다.
# dt.weekday는 '월요일'=0, '일요일'=6을 반환하므로, +1을 해주어 '월요일'=1, '일요일'=7로 변환합니다.
# 참고: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html
data['요일'] = data['일자'].dt.weekday + 1

In [8]:
data['요일']

0       7
1       1
2       2
3       3
4       4
       ..
8613    5
8614    6
8615    7
8616    1
8617    2
Name: 요일, Length: 8618, dtype: int32

In [9]:
# 4. 데이터 필터링
# '성산로(금화터널)' 지점에서 '유입' 방향으로 들어오는 데이터만을 선택합니다.
# 데이터를 먼저 필터링하여 분석에 필요한 데이터만 남깁니다.
# Boolean indexing을 사용하여 조건에 맞는 데이터만 필터링할 수 있습니다.
# 예: filtered_data = 데이터프레임[데이터프레임['컬럼명'] == '값' & 데이터프레임['컬럼명'] == '값']
# 참고: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#boolean-indexing
filtered_data = data[(data['지점명'] == '성산로(금화터널)') & (data['방향'] == '유입')]

In [10]:
filtered_data.head()

,일자,요일,지점명,지점번호,방향,구분,0시,1시,2시,3시,...,14시,15시,16시,17시,18시,19시,20시,21시,22시,23시
0,2023-01-01,7,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,564.0,414.0,268.0,208.0,...,1388.0,1370.0,1322.0,1263.0,979.0,875.0,785.0,824.0,552.0,421.0
1,2023-01-02,1,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,294.0,190.0,146.0,134.0,...,1459.0,1571.0,1566.0,1792.0,1582.0,1248.0,997.0,973.0,872.0,616.0
2,2023-01-03,2,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,419.0,271.0,203.0,177.0,...,1663.0,1678.0,1711.0,1851.0,1697.0,1265.0,1081.0,1090.0,883.0,757.0
3,2023-01-04,3,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,502.0,335.0,247.0,192.0,...,1679.0,1798.0,1719.0,1807.0,1679.0,1240.0,1112.0,1069.0,909.0,769.0
4,2023-01-05,4,성산로(금화터널),A-01,유입,봉원고가차도->독립문역,523.0,334.0,235.0,196.0,...,1530.0,1627.0,1805.0,1798.0,1770.0,1289.0,1152.0,1111.0,1030.0,756.0


In [11]:
filtered_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 31 entries, 0 to 30
Data columns (total 30 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   일자      31 non-null     datetime64[ns]
 1   요일      31 non-null     int32         
 2   지점명     31 non-null     object        
 3   지점번호    31 non-null     object        
 4   방향      31 non-null     object        
 5   구분      31 non-null     object        
 6   0시      31 non-null     float64       
 7   1시      31 non-null     float64       
 8   2시      31 non-null     float64       
 9   3시      31 non-null     float64       
 10  4시      31 non-null     float64       
 11  5시      31 non-null     float64       
 12  6시      31 non-null     float64       
 13  7시      31 non-null     float64       
 14  8시      31 non-null     float64       
 15  9시      31 non-null     float64       
 16  10시     31 non-null     float64       
 17  11시     31 non-null     float64       
 18  12시     31 non-nu

In [12]:
# 5. 피처와 타깃 값 설정
# '요일'을 독립 변수로, '0시' 교통량을 종속 변수로 설정합니다.
# 참고: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html
X = filtered_data[['요일']]
y = filtered_data['0시']

In [13]:
X.head()

,요일
0,7
1,1
2,2
3,3
4,4


In [14]:
y.head()

0    564.0
1    294.0
2    419.0
3    502.0
4    523.0
Name: 0시, dtype: float64

In [15]:
# 6. 학습 데이터와 테스트 데이터 분할
# 학습 데이터와 테스트 데이터를 나누는 함수로 train_test_split을 사용합니다.
# 참고: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [16]:
# 7. 선형 회귀 모델 정의 및 학습
# scikit-learn의 LinearRegression 모델을 사용하여 학습 데이터를 학습시킵니다.
# 참고: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html
model = LinearRegression()
model.fit(X_train, y_train)

,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


In [17]:
# 8. 모델 평가
# 학습된 모델을 평가하기 위해 R² 점수를 사용합니다. 1에 가까울수록 좋은 성능을 의미합니다.
# scikit-learn의 score() 함수는 기본적으로 R² 점수를 반환합니다.
# 참고: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn.linear_model.LinearRegression.score
r2_score = model.score(X_test, y_test)
print(f"R²: {r2_score}")

R²: 0.2835616050573251
